# 3D Structure from overlapping 2D images

## Authors: Muneeb Aadil, Sibt Ul Hussain

### Tutorial adapted to Google Colab from: https://github.com/muneebaadil/how-to-sfm/blob/master/tutorial/tutorial.ipynb

In [0]:
# remove any unnecessary colab folders
# !rm -rf <folder_name>

In [0]:
# requirements
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16
!pip install open3d-python

# data
!apt-get install subversion
!svn checkout "https://github.com/jojker/PML_Workshops/trunk/Summer 2019/Day 2 - Goal 1 - Turning Images into Data/Ex 4 - 3D structure from overlapping 2D images/Data"

# move two needed python files (utils.py and SfM.py) to the current directory
!mv /content/Data/utils.py /content
!mv /content/Data/SfM.py /content


# Chapter 2. Epipolar Geometry


In [0]:
%matplotlib inline

# loading needed libraries
import utils as ut 
import SfM as sfmnp
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import cv2
import numpy as np
import open3d as o3d

path = "/content/Data/images/"

In [0]:
# run this cell to use plotly in google colab
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

##Reading a pair of images, and comparing SIFT matches

In [0]:
# Reading two images for reference
img1 = cv2.imread(path + 'fountain3.jpg')
img2 = cv2.imread(path + 'fountain5.jpg')

# Converting from BGR to RGB format
img1 = img1[:,:,::-1]
img2 = img2[:,:,::-1]

# NOTE: you can adjust appropriate figure size according to the size of your screen
fig,ax=plt.subplots(ncols=2,figsize=(14,7)) 
ax[0].imshow(img1)
ax[1].imshow(img2)

**keypoints**  --  spatial locations, or points in an image that define what is interesting or stands out in the image. Keypoints are special because no matter how the image changes (rotation, scaling, translation, or distortion through projective transformation) you should be able to find the same keypoints in the modified image as the original image.

example of keypoints in two images where one is rotated - the same points are found in both images and are connected

<img src=https://i.stack.imgur.com/L4RUT.png width="700">


What makes keypoints different between frameworks is the way you describe these keypoints. These are known as **descriptors**. Each detected keypoint has an associated descriptor that accompanies it. Some frameworks only do a keypoint detection, while other frameworks are simply a description framework. SIFT and SURF are examples of frameworks that both detect and describe the keypoints.

SIFT  --   scale-invariant feature transform (feature detection algorithm used in computer vision)

SURF  --  speeded up robust features (feature detection and descriptor algorithm in computer vision)

Descriptors are primarily concerned with both the scale and the orientation of the keypoint. Descriptor are needed to match keypoints in different images.

An example descriptor is keypoint orientation. For an origientation descriptor, the algorithm searches the image pixels near the keypoint for the most dominant orientation of the gradient angles in the patch


In [0]:
# Getting SIFT/SURF features for image matching (this might take a few minutes)

# kp1 and kp2 are the keypoints of image 1 and 2 found using the cv2.xfeatures2d.SURF_create() function inside utils.py
# desc1 and desc2 are the descriptors of image 1 and 2 found using the cv2.xfeatures2d.SURF_create() function inside utils.py

kp1,desc1,kp2,desc2,matches=ut.GetImageMatches(img1,img2)

# Aligning two keypoint vectors by 
# 1) retrieving the corresponding indices of keypoints (img1idx and img2idx)
# 2) filtering out the keypoints that were NOT matched and 
# 3) retreiving the image coordinates of matched keypoints (img1pts and img2pts)

img1pts,img2pts,img1idx,img2idx=ut.GetAlignedMatches(kp1,desc1,kp2,desc2,matches)

## Fundamental Matrix Computation

The **fundamental matrix** relates corresponding points between a pair of uncalibrated images. The matrix transforms homogeneous image points in one image to epipolar lines in the other image.

**epipolar line**  --  straight line of intersection of the epipolar plane with the image plane. It is the image in one camera of a ray through the optical centre and image point in the other camera. All epipolar lines intersect at the epipole.

A point x in one image generates a line in the other on which its corresponding point x' must lie. We see that the search for correspondences is thus reduced from a region to a line. The corresponding point for x must lie on the epipolar line.

<img src=http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/OWENS/LECT10/img17.gif width="500">

See the following link for additiona linformation on epipolar geometry and triangulation

http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/OWENS/LECT10/node3.html


### Eight Point Algorithm

In [0]:
# select the first 8 matched keypoints for an eight point algorithm
img1pts_, img2pts_ = img1pts[:8], img2pts[:8] # underscores used to avoid conflict between python variables

# determine the fundamental matrix using opencv with the 8 point algorithm (cv2.FM_8POINT)
Fgt, mask = cv2.findFundamentalMat(img1pts_,img2pts_,method=cv2.FM_8POINT)

# in house determination of fundamental matrix inside SfM.py file
F = sfmnp.EstimateFundamentalMatrix(img1pts_,img2pts_)

The in house methods are included here in the additional files, SfM.py and utils.py, so you can see the details of the calculations.

In [0]:
# check that the in house detmerination and the opencv determination of the fundamental matrix are similar
print(F,'\n\n',Fgt)

# gives an error if the two variables F and Fgt are not similar enough (rtol - relative tolerance, atol - absolute tolerance)
np.testing.assert_allclose(F, Fgt, rtol=1e-3,atol=1e-2)

### Normalized 8 Point Algorithm

In [0]:
# in house determination of normalized fundamental matrix
F_normalized = sfmnp.EstimateFundamentalMatrixNormalized(img1pts_,img2pts_)
print(F_normalized)

### with RANSAC (Random Sample Consensus)

The RANSAC algorithm allows for more than 8 keypoints to be used to determine the fundamental matrix

Random sample consensus (RANSAC) is an iterative method to estimate parameters of a mathematical model from a set of observed data that contains outliers, when outliers are to be accorded no influence on the values of the estimates

In [0]:
# calculate the fundamental matrix using the RANSAC algorithm
Fgt, maskgt = cv2.findFundamentalMat(img1pts,img2pts,method=cv2.FM_RANSAC)

# The mask is an output array, every element of which is set to 0 for outliers and to 1 for the other points. 
# The array is computed only in the RANSAC and LMedS methods.

# convert to boolean (true/false)
maskgt = maskgt.astype(bool).flatten()

# in house estimation of the fundamental matrix using the RANSAC method
F, mask = sfmnp.EstimateFundamentalMatrixRANSAC(img1pts,img2pts,.1,iters=20000)

In [0]:
# check that the two methods are similar
print(F,'\n\n',Fgt)
np.testing.assert_allclose(F, Fgt, rtol=1e-3,atol=1e-3)

## Epipolar Lines Computation

In [0]:
# determine the epipolar lines
lines2=sfmnp.ComputeEpiline(img1pts[maskgt],1,Fgt)
lines1=sfmnp.ComputeEpiline(img2pts[maskgt],2,Fgt)

## Visualizations I: Epipolar Geometry
### Epipolar Lines

In [0]:
# drawing epipolar lines and keypoints with cv2.lines() and cv2.circle()

num_to_draw = 25  # number of lines and points to draw

# returns new images with the lines and points drawn on
tup = ut.drawlines(img2,img1,lines2,img2pts[maskgt],img1pts[maskgt],drawOnly=num_to_draw,
                   linesize=10,circlesize=35)

# making the images with the lines and points viewable with imshow() in matplotlib
epilines2 = np.concatenate(tup[::-1],axis=1) #reversing the order of left and right images

# show the first set of points and lines
plt.figure(figsize=(9,4))
plt.imshow(epilines2)


# repeat for the second image

tup = ut.drawlines(img1,img2,lines1,img1pts[maskgt],img2pts[maskgt],drawOnly=num_to_draw,
                   linesize=10,circlesize=35)
epilines1 = np.concatenate(tup,axis=1) 

plt.figure(figsize=(9,4))
plt.imshow(epilines1)

## Pose Estimation

Geometric camera calibration, also referred to as camera resectioning, estimates the parameters of a lens and image sensor of an image or video camera. You can use these parameters to correct for lens distortion, measure the size of an object, or determine the location of the camera in the scene. These tasks are used in applications such as machine vision to detect and measure objects. 

<img src=https://www.mathworks.com/help/vision/ug/calibration_applications.png width="500">

The camera calibration matrix is a 3x3 or 3x4 matrix containing 5 parameters encompassing focal length, image sensor format, and the principal point

focal length - distance of convergence of light through a lens

image sensor fomat - shape and size of the image sensor

principal point - A point R at the intersection of the optical axis and the image plane (ideally the image center)

<img src=https://wikimedia.org/api/rest_v1/media/math/render/svg/e712d4b11c471fba8d70b4ae2f8790328040c6fd width="300">

The parameters $ \alpha _x = f \cdot m _x $ and $ \alpha _y = f \cdot m _y $ represent focal length in terms of pixels, where $ m_ x $ and $ m_ y $ are the scale factors relating pixels to distance and $ f $ is the focal length. $ \gamma $ represents the skew coefficient between the x and the y axis, and is often 0. $ u _0 $ and $ v _0 $ is the principal point

Given a point in one image the  **essential matrix** tells us which epipolar line to search along in the second view

In [0]:
# known camera calibration matrix
K = np.array([[2759.48,0,1520.69],[0,2764.16,1006.81],[0,0,1]])

# determine the essential matrix, E
E = K.T.dot(Fgt.dot(K))

# determining the camera pose possible locations.
# R is the 3x3 rotation matrix to convert from world to camera coordinate system
# t is the 3x1 translation vector from the camer'as origin to the world's origin
R1,R2,t = sfmnp.ExtractCameraPoses(E)
t = t[:,np.newaxis]

## Visualizations II: Camera Poses

There are several possible solutions following the reconstruction from the essential matrix.

<img src=https://lh3.googleusercontent.com/-CN6u5U6R4u0/WECN2oY-HJI/AAAAAAAAAz8/Fhzukgas4u0MuswsKDXz564GLHTIPTnbACLcB/s1600/geometry.jpg width="400">



In [0]:
for R_ in [R1,R2]: 
    for t_ in [t,-t]:
        
        fig = plt.figure(figsize=(9,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

        ut.PlotCamera(np.eye(3,3),np.zeros((3,)),ax)
        ut.PlotCamera(R_,t_[:,0],ax)

# 3D Scene Estimations

## Triangulation: Direct Linear Transform (DLT) Method

The most commonly used camera calibration method is perhaps the DLT (direct linear
transformation) method originally reported by Abdel-Aziz and Karara [1]. The DLT method uses a
set of control points whose object space/plane coordinates are already known. The control points
are normally fixed to a rigid frame, known as the calibration frame. The problem is essentially to
calculate the mapping between the 2D image space coordinates (xi
) and the 3D object space
coordinates (Xi
). For this 3D 2D correspondence the mapping should take the form of a 3x4
projection matrix (P) such that xi
 = PXi
 for all i. 
 
[1] Abdel-Aziz, Karara., Direct linear transformation into object space coordinates in closerange photogrametry. In Proc. Symp. Close-Range Photogrametry, 1971: p. 1-18. 

In [0]:
def GetTriangulatedPts(img1pts,img2pts,K,R,t,triangulateFunc): 
    img1ptsHom = cv2.convertPointsToHomogeneous(img1pts)[:,0,:]
    img2ptsHom = cv2.convertPointsToHomogeneous(img2pts)[:,0,:]

    img1ptsNorm = (np.linalg.inv(K).dot(img1ptsHom.T)).T
    img2ptsNorm = (np.linalg.inv(K).dot(img2ptsHom.T)).T

    img1ptsNorm = cv2.convertPointsFromHomogeneous(img1ptsNorm)[:,0,:]
    img2ptsNorm = cv2.convertPointsFromHomogeneous(img2ptsNorm)[:,0,:]
    
    pts4d = triangulateFunc(np.eye(3,4),np.hstack((R,t)),img1ptsNorm.T,img2ptsNorm.T)
    pts3d = cv2.convertPointsFromHomogeneous(pts4d.T)[:,0,:]
    
    return pts3d

In [0]:
pts3dgt = GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R2,t,cv2.triangulatePoints)
pts3d = GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R2,t,sfmnp.Triangulate)

In [0]:
# check the in house vs cv2 methods
print(pts3dgt[:5],'\n\n',pts3d[:5])
np.testing.assert_allclose(pts3d,pts3dgt,rtol=1e-2,atol=1e-1)

In [0]:
# placing the data into a convenient form
configSet = [None,None,None,None]
configSet[0] = (R1,t,GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R1,t,cv2.triangulatePoints))
configSet[1] = (R1,-t,GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R1,-t,cv2.triangulatePoints))
configSet[2] = (R2,t,GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R2,t,cv2.triangulatePoints))
configSet[3] = (R2,-t,GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,R2,-t,cv2.triangulatePoints))

## Visualizing the triangulated points of configurations

In [0]:
for cs in configSet: 
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    ut.PlotCamera(np.eye(3,3),np.zeros((3,)),ax,scale=5,depth=5)
    ut.PlotCamera(cs[0],cs[1][:,0],ax,scale=5,depth=5)

    pts3d = cs[-1]
    ax.scatter3D(pts3dgt[:,0],pts3dgt[:,1],pts3dgt[:,2])

    ax.set_xlim(left=-50,right=50)
    ax.set_ylim(bottom=-50,top=50)
    ax.set_zlim(bottom=-20,top=20)

## Camera Pose Disambiguation

There are multiple camera poses that solve the triangulation methods. Here we select the correct camera pose by counting the number of points in front of each camera for each pose. The pose with the most points is correct

In [0]:
# determine the correct camera pose transformation matrix elements R and t
_,Rgt,tgt,mask2=cv2.recoverPose(E,img1pts[maskgt],img2pts[maskgt],K)
R,t,count = sfmnp.DisambiguateCameraPose(configSet)

In [0]:
# comparing the in house and opencv methods
print(R,'\n\n',Rgt,'\n\n',t,'\n\n',tgt)
np.testing.assert_allclose(R,Rgt,rtol=1e-7,atol=1e-4)
np.testing.assert_allclose(t,tgt,rtol=1e-7,atol=1e-4)

## Visualizing Point Cloud Output

In [0]:
# build the construction into a point cloud (x,y,z) and save it
pts3d = GetTriangulatedPts(img1pts[maskgt],img2pts[maskgt],K,Rgt,tgt,cv2.triangulatePoints)
ut.pts2ply(pts3d,'fountain_2view.ply')

### this section could use some work. Currently the RGB color of the point is not taken. However,
### point clouds often have associated RGB with each point. One recommendation would be to take one
### or an average of the RGB values of the pixel that is being triangulated.  MeshLab is a convenient
### program to view RGB associated point clouds. 

In [0]:
# read the point clouid with open3d and plot using plotly
pcd = o3d.io.read_point_cloud("fountain_2view.ply")

print(pcd)

points = np.asarray(pcd.points)

# remove poitns that are clearly outside the image location
keep_points = points
num_deleted = 0
for i in range(len(points)):
  if np.abs(points[i][0]) > 3:
    keep_points = np.delete(keep_points, i-num_deleted, axis=0)
    num_deleted = num_deleted+1

x = keep_points[:,0]
y = keep_points[:,1]
z = keep_points[:,2]


# allow for colab to use plotly for interactive plotting 
configure_plotly_browser_state()

import plotly
from plotly.graph_objs import Scatter3d, Layout

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=4))],
    "layout": Layout(title="fountain 3D reconstruction from 2 images")
})

# Reprojection Error: Evaluation

## Computation

It is convenient to calculate the error in the image points and the reprojected points

Here we reproduce the points as they are in the image from the points calculated in the world coordinate system from the rotation matrix, the translation vector, and the camera calibration matrix.

In [0]:
"""
ComputeReprojections(X,R,t,K)

    inputs
    X: (n,3) 3D triangulated points in world coordinate system
    R: (3,3) Rotation Matrix to convert from world to camera coordinate system
    t: (3,1) Translation vector (from camera's origin to world's origin)
    K: (3,3) Camera calibration matrix
    
    output: (n,2) Projected points into image plane
"""
img1ptsReproj = sfmnp.ComputeReprojections(pts3d,np.eye(3,3),np.zeros((3,1)),K)
img2ptsReproj = sfmnp.ComputeReprojections(pts3d,Rgt,tgt,K)

In [0]:
# calculate the error in the image points compared to the reprojected image points
err2 = sfmnp.ComputeReprojectionError(img2pts[maskgt], img2ptsReproj)
err1 = sfmnp.ComputeReprojectionError(img1pts[maskgt], img1ptsReproj)

err1, err2

## Visualizations

In [0]:
# draw a true point with an x and a reprojected point as a dot (they are nearly on top of one another)

fig,ax=plt.subplots(figsize=(9,5))
ut.DrawCorrespondences(img1,img1pts[maskgt],img1ptsReproj,ax)

fig,ax=plt.subplots(figsize=(9,5))
ut.DrawCorrespondences(img2,img2pts[maskgt],img2ptsReproj,ax)

# Perspective-n-Point Algorithm: New Camera Registration

## Reading third image and 2D-3D Matching using SIFT

In [0]:
# read in a 3rd image of the fountain
img3 = cv2.imread(path + 'fountain7.jpg')
img3 = img3[:,:,::-1]

# determine the keypoints and descriptors for the 3rd image
surfer=cv2.xfeatures2d.SURF_create()
kp3, desc3 = surfer.detectAndCompute(img3,None)

# show the 3rd image
plt.figure()
plt.imshow(img3)

# find the points that match between all three image keypoints
img3pts,pts3dpts = ut.Find2D3DMatches(desc1,img1idx,desc2,img2idx,desc3,kp3,maskgt,pts3d)

## Perspective-n-Point (PnP) Algorithm

Perspective-n-Point is the problem of estimating the pose of a calibrated camera given a set of n 3D points in the world and their corresponding 2D projections in the image. The camera pose consists of 6 degrees-of-freedom (DOF) which are made up of the rotation (roll, pitch, and yaw) and 3D translation of the camera with respect to the world.

### RANSAC solution

The RANSAC algorithm allows for more than 8 keypoints to be used to determine the fundamental matrix

Random sample consensus (RANSAC) is an iterative method to estimate parameters of a mathematical model from a set of observed data that contains outliers, when outliers are to be accorded no influence on the values of the estimates

In [0]:
# determine the new rotation matrix, translation vector, and mask
retval,Rvec,tnew,mask3gt = cv2.solvePnPRansac(pts3dpts[:,np.newaxis],img3pts[:,np.newaxis],
                                            K,None,confidence=.99,flags=cv2.SOLVEPNP_DLS)
Rnew,_=cv2.Rodrigues(Rvec)

## Visualizations

In [0]:
# visualize the camera positions
fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ut.PlotCamera(np.eye(3,3),np.zeros((3,)),ax)
ut.PlotCamera(R,t[:,0],ax)
ut.PlotCamera(Rnew,tnew[:,0],ax,faceColor='red')

# Re-triangulation

In [0]:
kpNew, descNew = kp3, desc3

kpOld,descOld = kp1,desc1
ROld, tOld = np.eye(3), np.zeros((3,1))

accPts = []
for (ROld, tOld, kpOld, descOld) in [(np.eye(3),np.zeros((3,1)), kp1,desc1),(Rgt,tgt,kp2,desc2)]: 
    
    #Matching between old view and newly registered view.. 
    print('[Info]: Feature Matching..')
    matcher = cv2.BFMatcher(crossCheck=True)
    matches = matcher.match(descOld, desc3)
    matches = sorted(matches, key = lambda x:x.distance)
    imgOldPts, imgNewPts, _, _ = ut.GetAlignedMatches(kpOld,descOld,kpNew,
                                                          descNew,matches)
    
    #Pruning the matches using fundamental matrix..
    print('[Info]: Pruning the Matches..')
    F,mask=cv2.findFundamentalMat(imgOldPts,imgNewPts,method=cv2.FM_RANSAC)
    mask = mask.flatten().astype(bool)
    imgOldPts=imgOldPts[mask]
    imgNewPts=imgNewPts[mask]
    
    #Triangulating new points
    print('[Info]: Triangulating..')
    newPts = sfmnp.GetTriangulatedPts(imgOldPts,imgNewPts, K, Rnew,tnew,cv2.triangulatePoints,ROld,tOld)
    
    #Adding newly triangulated points to the collection
    accPts.append(newPts)

# Final Result 

In [0]:
#Adding the original 2-view-sfm point cloud and saving the whole collection
accPts.append(pts3d)
ut.pts2ply(np.concatenate((accPts),axis=0),'fountain_nview.ply')

In [0]:
# read the point cloud with open3d and display with plotly
pcd = o3d.io.read_point_cloud("fountain_nview.ply")

print(pcd)

points = np.asarray(pcd.points)

# remove poitns that are clearly outside the image location
keep_points = points
num_deleted = 0
for i in range(len(points)):
  if np.abs(points[i][0]) > 3:
    keep_points = np.delete(keep_points, i-num_deleted, axis=0)
    num_deleted = num_deleted+1

x = keep_points[:,0]
y = keep_points[:,1]
z = keep_points[:,2]

# allows for colab to use plotly for interactive plotting 
configure_plotly_browser_state()

import plotly
from plotly.graph_objs import Scatter3d, Layout

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=3))],
    "layout": Layout(title="fountain 3D reconstruction from 3 images")
})

# Remaining Challenges
1. Create a point cloud that uses the RGB pixel values in addition to the traingulated (x,y,z) points and plot it as an interactive plot
2. Implement non local histogram normalization
3. Use more than 3 images to create the point cloud
4. Use your own images to build a 3D point cloud